In [10]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from database.weather import Weather
from transformer.date_transformer import DateTransformer
from transformer.column_transformer import ColumnTransformer
from transformer.model_transformer import ModelTransformer
from transformer.product_transformer import ProductTransformer
from transformer.predictor_transformer import PredictorTransformer
from preprocessor.model_preprocessor import ModelPreprocessor
from preprocessor.predictor_preprocessor import PredictorPreprocessor
from modeler.modeler import Modeler as sp
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math

In [14]:
## Loading Constants
start = "2008-01-01"
end = datetime(2020,10,1).strftime("%Y-%m-%d")
# Loading Databases
strat_db = Strategy("unity")
market = Market()
weather = Weather()
sec = SEC("sec")
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
market.close()
reload = True
quarterly_range = range(1,5)
yearly_range = range(2014,2021)
model_range = range(len(tickers["Symbol"]))
datasets = [
#             "pdr"
#             ,"tiingo"
            "finnhub"
           ]

In [15]:
sims = []
training_days = 100
gap = 365
quarter_gap = int(gap/90)
training_years = 4
yearly_gap = int(gap/365)
sims = []
sec.connect()
strat_db.connect()
market.connect()
for dataset in datasets:
    quarterly_classification_data = market.retrieve_data("dataset_{}_quarter_classification".format(dataset))
    quarterly_regression_data = market.retrieve_data("dataset_{}_quarter_regression".format(dataset))
    strat_db.drop_table("{}_price_quarterly_sim".format(dataset))
    strat_db.drop_table("{}_price_quarterly_sim_errors".format(dataset))
    for year in tqdm(yearly_range):
        for quarter in tqdm(quarterly_range):
            try:
                ## Setting Up
                print(year,quarter)
                date_ranges = DateUtils.create_quarterly_training_range_rec(year,quarter,training_years,gap)
                training_start,training_end,prediction_start,prediction_end = date_ranges
                dates = pd.to_datetime(date_ranges)
                quarters = [x.quarter for x in dates]
                years = [x.year for x in dates]
                print(date_ranges)
                ### switch to all tickers
                for i in model_range:
                    try:
                        ticker = tickers.iloc[i]["Symbol"].replace(".","-")
                        if ticker in quarterly_regression_data.columns:
                            price = market.retrieve_price_data("{}_prices".format(dataset),ticker)
                            if dataset == "pdr":
                                price = ColumnTransformer.rename_columns(price," ")
                            else:
                                price = ColumnTransformer.rename_columns(price,"")
                            price = DateTransformer.convert_to_date(dataset,price,"date")
                            mt = ModelTransformer(ticker,training_start,training_end,gap)
                            pt = ModelTransformer(ticker,prediction_start,prediction_end,gap)
                            mr = ModelPreprocessor(ticker)
                            prot = ProductTransformer(ticker,prediction_start,prediction_end)
                            ## regression_model
                            rqpd = mt.quarterly_price_transform(quarterly_regression_data,ticker
                                                                                ,years[0],quarters[0],
                                                                               years[1],quarters[1],quarter_gap)
                            rpd = mr.day_trade_preprocess_regression(rqpd.copy(),ticker)
                            rpr = sp.regression(rpd,ranked=False,tf=False)
                            ## classification_model
                            cqpd = mt.quarterly_price_transform(quarterly_classification_data,ticker
                                                                                ,years[0],quarters[0],
                                                                               years[1],quarters[1],quarter_gap)
                            cpd = mr.day_trade_preprocess_classify(cqpd.copy(),ticker)
                            cpr = sp.classification(cpd,tf=False)
                            fundamental_results = pd.DataFrame([cpr,rpr])
                            product_refineds = []
                            product_qpds = []
                            for i in range(len(fundamental_results)):
                                fundamental_result = fundamental_results.iloc[i]
                                if fundamental_result["model_type"] == "regression":
                                    fundamental_data = pt.quarterly_price_transform_product(quarterly_regression_data,ticker
                                                                                ,years[2],quarters[2],
                                                                               years[3],quarters[3],quarter_gap)
                                    product_refined = mr.day_trade_preprocess_regression(fundamental_data.copy(),ticker)
                                else:
                                    fundamental_data = pt.quarterly_price_transform_product(quarterly_classification_data,ticker
                                                                                ,years[2],quarters[2],
                                                                               years[3],quarters[3],quarter_gap)
                                    product_refined = mr.day_trade_preprocess_classify(fundamental_data.copy(),ticker)
                                product_refineds.append(product_refined)
                            sim = prot.merge_quarterlies(product_refineds,fundamental_results,"price",yearly_gap)
                            strat_db.store_data("{}_price_quarterly_sim".format(dataset),pd.DataFrame([sim]))
                    except Exception as e:
                        message = {"status":"quarterly modeling","ticker":ticker,"year":str(year),"quarter":str(quarter),"message":str(e)}
                        print(message)
                        strat_db.store_data("price_quarterly_sim_errors",pd.DataFrame([message]))
            except Exception as e:
                print(year,quarter,str(e))
sec.close()
market.close()
strat_db.close()

  0%|                                                                                                                                                                                              | 0/4 [00:00<?, ?it/s]

2014 1
['2010-01-01', '2013-12-31', '2014-01-01', '2014-03-31']
{'status': 'quarterly modeling', 'ticker': 'ABBV', 'year': '2014', 'quarter': '1', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'ADP', 'year': '2014', 'quarter': '1', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'ALL', 'year': '2014', 'quarter': '1', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'AON', 'year': '2014', 'quarter': '1', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'AOS', 'year': '2014', 'quarter': '1', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'APD', 'year': '2014', 'quarter': '1', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'ARE', 'year': '2014', 'quarter': '1', 'message': 'single 


 25%|████████████████████████████████████████████▌                                                                                                                                     | 1/4 [28:04<1:24:12, 1684.19s/it]

{'status': 'quarterly modeling', 'ticker': 'ZBRA', 'year': '2014', 'quarter': '1', 'message': "'t'"}
{'status': 'quarterly modeling', 'ticker': 'ZION', 'year': '2014', 'quarter': '1', 'message': "'t'"}
{'status': 'quarterly modeling', 'ticker': 'ZTS', 'year': '2014', 'quarter': '1', 'message': "'t'"}
2014 2
['2010-04-01', '2014-03-31', '2014-04-01', '2014-06-30']
{'status': 'quarterly modeling', 'ticker': 'ADP', 'year': '2014', 'quarter': '2', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'AON', 'year': '2014', 'quarter': '2', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'APD', 'year': '2014', 'quarter': '2', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'ATO', 'year': '2014', 'quarter': '2', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'ATVI', 'year': '2014', 'quarter': 


 50%|██████████████████████████████████████████████████████████████████████████████████████████                                                                                          | 2/4 [56:04<56:06, 1683.08s/it]

...

{'status': 'quarterly modeling', 'ticker': 'ZBRA', 'year': '2019', 'quarter': '3', 'message': "'t'"}
{'status': 'quarterly modeling', 'ticker': 'ZION', 'year': '2019', 'quarter': '3', 'message': "'t'"}
{'status': 'quarterly modeling', 'ticker': 'ZTS', 'year': '2019', 'quarter': '3', 'message': "'t'"}
2019 4
['2015-10-01', '2019-09-30', '2019-10-01', '2019-12-31']
{'status': 'quarterly modeling', 'ticker': 'ABC', 'year': '2019', 'quarter': '4', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'ADP', 'year': '2019', 'quarter': '4', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'BAC', 'year': '2019', 'quarter': '4', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'BIIB', 'year': '2019', 'quarter': '4', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'CDW', 'year': '2019', 'quarter': 


  0%|                                                                                                                                                                                              | 0/4 [00:00<?, ?it/s]

{'status': 'quarterly modeling', 'ticker': 'ZBRA', 'year': '2019', 'quarter': '4', 'message': "'t'"}
{'status': 'quarterly modeling', 'ticker': 'ZION', 'year': '2019', 'quarter': '4', 'message': "'t'"}
{'status': 'quarterly modeling', 'ticker': 'ZTS', 'year': '2019', 'quarter': '4', 'message': "'t'"}
2020 1
['2016-01-01', '2019-12-31', '2020-01-01', '2020-03-31']
{'status': 'quarterly modeling', 'ticker': 'ABC', 'year': '2020', 'quarter': '1', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'ADP', 'year': '2020', 'quarter': '1', 'message': 'single positional indexer is out-of-bounds'}


 86%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                         | 6/7 [11:25:33<1:54:15, 6855.53s/it]


KeyboardInterrupt: 

In [13]:
sim

{'year': 2015,
 'quarter': 1,
 'quarterly_price_classification_prediction': 0.0,
 'quarterly_price_classification_score': 0.6666666666666666,
 'quarterly_price_regression_prediction': 829.25635,
 'quarterly_price_regression_score': 97.78495649957767,
 'ticker': 'AAL'}